In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# Transformer Encoder for time series data
from src.models.transformer_ts_tf import Transformer

2023-10-12 16:07:24.829366: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 16:07:24.833961: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-12 16:07:24.936287: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-12 16:07:24.938441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 16:07:26.647058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Data preprocessing

In [3]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [5]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [6]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

## Sequence helper

In [7]:
def generate_sequences(dataset, timesteps):
    dataset["x_train"], dataset["y_train"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_train"], columns=x.columns),
                dataset["y_train"],
            ],
            axis=1,
        ).values,
        timesteps,
    )

    dataset["x_test"], dataset["y_test"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_test"], columns=x.columns),
                dataset["y_test"],
            ],
            axis=1,
        ).values,
        timesteps,
    )
    return dataset

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [8]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    estimator_params=None,
    imputer_params=None,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [9]:
class TransformerWrapper(RegressorMixin):
    def __init__(self, params):
        self.params = params
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = Transformer(
            num_hid=self.params["num_features"],
            time_steps=self.params["timesteps"],
            num_head=self.params["num_heads"],
            num_layers_enc=self.params["num_layers_enc"],
            num_feed_forward=self.params["num_feed_forward"],
        )
        model.compile(
            tf.keras.optimizers.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [10]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [11]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [12]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [13]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Transformer",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [14]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cem_c.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [15]:
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm",  # Maybe pH of the immersion liquid | pH (acidity or alkalinity)
        "Initial Setting Time",
        "flow",  # Maybe flow table test
        "residuo 24 micron",  # Maybe the residue left after passing the material through a sieve
        "R_wp",  # Maybe water to powder ratio
        "CS2",  # 2-day Compressive Strength
        "Soundness",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [16]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

# Transformer Encoder

# Configs:

1. TIMESTEPS: 1, 7, 14

2. Epochs: 300

3. Batch size: 16

4. num heads: 1

5. FF Layers enc: 1

      5.1 Units 16

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-3

In [17]:
params = {}
params["timesteps"] = None
params["num_features"] = None
params["num_heads"] = 1
params["num_layers_enc"] = 1
params["num_feed_forward"] = 16

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [18]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"], axis=1)
    y = df_copy["CS28"]

    params["timesteps"] = timesteps
    params["num_features"] = x.shape[-1]

    scores = custom_cross_validate(
        TransformerWrapper,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        estimator_params={"params": params},
        imputer_params={"strategy": "median"},
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-10-12 16:07:30.259765: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-12 16:07:30.259828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-10-12 16:07:30.259843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-10-12 16:07:30.260056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-10-12 16:07:30.260111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-10-12 16:07:30.260120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.272 (0.099)
MAE: 1.017 (0.082)
MAPE: 0.025 (0.002)
R2: 0.788 (0.033)


******
[TEST]
******
RMSE: 2.348 (0.205)
MAE: 1.877 (0.157)
MAPE: 0.047 (0.004)
R2: 0.251 (0.177)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.368 (0.112)
MAE: 1.090 (0.114)
MAPE: 0.027 (0.003)
R2: 0.752 (0.041)


******
[TEST]
******
RMSE: 3.927 (1.764)
MAE: 2.945 (0.741)
MAPE: 0.073 (0.018)
R2: -1.630 (3.833)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.535 (0.130)
MAE: 1.233 (0.119)
MAPE: 0.031 (0.003)
R2: 0.688 (0.059)


******
[TEST]
******
RMSE: 4.705 (2.081)
MAE: 3.644 (1.465)
MAPE: 0.090 (0.036)
R2: -2.358 (3.613)


Minutes Elapsed:  24.19182976881663


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    params["timesteps"] = timesteps

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            TransformerWrapper,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params={"params": params},
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.036 (0.152)
MAE: 1.652 (0.135)
MAPE: 0.042 (0.003)
R2: 0.236 (0.155)


******
[TEST]
******
RMSE: 3.115 (1.395)
MAE: 2.800 (1.335)
MAPE: 0.068 (0.028)
R2: -2.232 (1.738)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 2.154 (0.271)
MAE: 1.782 (0.212)
MAPE: 0.045 (0.005)
R2: 0.071 (0.221)


******
[TEST]
******
RMSE: 13.368 (10.315)
MAE: 11.076 (8.818)
MAPE: 0.271 (0.222)
R2: -197.558 (282.105)


Minutes Elapsed:  4.623471943537394


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [20]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None
    params["timesteps"] = timesteps

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            TransformerWrapper,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params={"params": params},
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.580 (0.304)
MAE: 1.278 (0.278)
MAPE: 0.032 (0.007)
R2: 0.654 (0.165)


******
[TEST]
******
RMSE: 3.062 (1.185)
MAE: 2.314 (0.720)
MAPE: 0.057 (0.019)
R2: -0.887 (1.684)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.608 (0.361)
MAE: 1.282 (0.319)
MAPE: 0.032 (0.008)
R2: 0.606 (0.262)


******
[TEST]
******
RMSE: 8.955 (6.283)
MAE: 7.044 (5.302)
MAPE: 0.174 (0.133)
R2: -20.190 (23.266)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.646 (0.245)
MAE: 1.294 (0.236)
MAPE: 0.032 (0.006)
R2: 0.609 (0.196)


******
[TEST]
******
RMSE: 12.507 (9.035)
MAE: 11.023 (8.432)
MAPE: 0.273 (0.211)
R2: -38.589 (50.178)


Minutes Elapsed:  14.835478806495667


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [21]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None
    params["timesteps"] = timesteps

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)

        # Sequence Splitting
        data_train = pd.concat(
            [
                pd.DataFrame(x_train, columns=x.columns),
                y_train.reset_index(drop=True),
            ],
            axis=1,
        ).values
        data_test = pd.concat(
            [
                pd.DataFrame(x_test, columns=x.columns),
                y_test.reset_index(drop=True),
            ],
            axis=1,
        ).values

        x_train, y_train = split_sequences(data_train, timesteps)
        x_test, y_test = split_sequences(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", TransformerWrapper(params))])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.314 (0.068)
MAE: 1.060 (0.064)
MAPE: 0.026 (0.002)
R2: 0.797 (0.021)


******
[TEST]
******
RMSE: 2.287 (0.118)
MAE: 1.875 (0.086)
MAPE: 0.047 (0.002)
R2: -0.411 (0.144)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.488 (0.128)
MAE: 1.207 (0.117)
MAPE: 0.030 (0.003)
R2: 0.737 (0.046)


******
[TEST]
******
RMSE: 2.618 (0.475)
MAE: 2.073 (0.348)
MAPE: 0.052 (0.009)
R2: -1.015 (0.745)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.521 (0.109)
MAE: 1.193 (0.111)
MAPE: 0.030 (0.003)
R2: 0.729 (0.038)


******
[TEST]
******
RMSE: 6.552 (2.588)
MAE: 5.431 (1.819)
MAPE: 0.139 (0.046)
R2: -13.548 (11.298)


Minutes Elapsed:  4.215300599733989


<IPython.core.display.Javascript object>

In [22]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params", "Timesteps"],
    dropna=False,
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features        Model            Cross Validation  \
                                                                        
0   Chemical + Mineralogical  Transformer  Blocking Time Series Split   
1   Chemical + Mineralogical  Transformer  Blocking Time Series Split   
2   Chemical + Mineralogical  Transformer           Out of time Split   
3   Chemical + Mineralogical  Transformer           Out of time Split   
4   Chemical + Mineralogical  Transformer           Out of time Split   
5   Chemical + Mineralogical  Transformer              Repeated KFold   
6   Chemical + Mineralogical  Transformer              Repeated KFold   
7   Chemical + Mineralogical  Transformer              Repeated KFold   
8   Chemical + Mineralogical  Transformer           Time Series Split   
9   Chemical + Mineralogical  Transformer           Time Series Split   
10  Chemical + Mineralogical  Transformer           Time Series Split   

                             Cross Validation Params Timesteps  RMSE Test  \
                                                                     mean   
0   {"N_Splits": 5, "Repeats": 3, "train_size": 0.8}         1   3.114550   
1   {"N_Splits": 5, "Repeats": 3, "train_size": 0.8}         7  13.367558   
2                                 {"Test Size": 0.2}         1   2.287286   
3                                 {"Test Size": 0.2}         7   2.618212   
4                                 {"Test Size": 0.2}        14   6.551835   
5                      {"N_Splits": 5, "Repeats": 3}         1   2.348438   
6                      {"N_Splits": 5, "Repeats": 3}         7   3.926786   
7                      {"N_Splits": 5, "Repeats": 3}        14   4.704696   
8            {"N_Splits": 5, "Repeats": 3, "Gap": 0}         1   3.061553   
9            {"N_Splits": 5, "Repeats": 3, "Gap": 0}         7   8.955276   
10           {"N_Splits": 5, "Repeats": 3, "Gap": 0}        14  12.506998   

                MAE Test           MAPE Test               R2 Test              
          std       mean       std      mean       std        mean         std  
0    1.394648   2.800128  1.334642  0.067639  0.028487   -2.231908    1.738361  
1   10.315332  11.076376  8.818495  0.270917  0.221775 -197.557816  282.104649  
2    0.117863   1.875133  0.085563  0.046745  0.002163   -0.411348    0.143551  
3    0.474872   2.073204  0.348208  0.051640  0.008832   -1.015463    0.745054  
4    2.588348   5.430754  1.818552  0.138549  0.045567  -13.548428   11.298344  
5    0.205177   1.877041  0.156928  0.046631  0.004014    0.250570    0.176752  
6    1.764122   2.945479  0.741069  0.073230  0.018164   -1.629605    3.832645  
7    2.081272   3.643621  1.464925  0.089861  0.036099   -2.358202    3.613376  
8    1.185395   2.314411  0.720080  0.056986  0.018807   -0.887137    1.683719  
9    6.282517   7.044018  5.302277  0.174462  0.132783  -20.189723   23.265840  
10   9.035115  11.023494  8.431691  0.273450  0.210898  -38.588646   50.177960

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [23]:
path = "../../../../../reports/results/local_models/partner_iv/cem_c/full/"
filename = "transformer_results_full_2.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [24]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/cem_c/grouped/"
filename = "transformer_results_grouped_2.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>